In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

**Note on this version**
* Include Titlle from Name
* Predict Age for Null Value
* Include Age Group
* Convert Fare to Log Fare
* Use Ticket Number to Add Group Information

**Data Cleaning**

In [3]:
#check data types
train.dtypes
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#Check unique rows
print('unique passengerID:', train.PassengerId.nunique())
print('number of rows: ', len(train))

unique passengerID: 891
number of rows:  891


In [5]:
#Check Null
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
#Handling Null Value
train_processed1=train.copy()
#drop columns with more than 50% null
train_processed1=train_processed1.loc[:,train_processed1.isna().mean() < 0.5]

In [7]:
#Get Title from Name
temp=pd.DataFrame()
i=0
for a in train_processed1['Name'].str.split(','):
    temp.at[i,'Title']=a[1].split('.')[0]
    i+=1
train_processed1=pd.concat([train_processed1,temp], axis=1) 

In [8]:
#Fill_in people in groups with same ticket 
temp = train_processed1.groupby('Ticket').count()[['PassengerId']]
temp.reset_index(level=0, inplace=True)
temp.columns=['Ticket','Groups']
train_processed1=pd.merge(train_processed1,temp,how='left',on='Ticket')

In [9]:
#Drop Name & Ticket
train_processed1=train_processed1.drop(columns=['Name','Ticket'])

In [10]:
train_processed2=train_processed1.copy()
train_processed2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Groups
0,1,0,3,male,22.0,1,0,7.2500,S,Mr,1
1,2,1,1,female,38.0,1,0,71.2833,C,Mrs,1
2,3,1,3,female,26.0,0,0,7.9250,S,Miss,1
3,4,1,1,female,35.0,1,0,53.1000,S,Mrs,2
4,5,0,3,male,35.0,0,0,8.0500,S,Mr,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,Rev,1
887,888,1,1,female,19.0,0,0,30.0000,S,Miss,1
888,889,0,3,female,NaN,1,2,23.4500,S,Miss,2
889,890,1,1,male,26.0,0,0,30.0000,C,Mr,1


In [11]:
#Fill Null Embarked by mode value
mode2=train_processed2['Embarked'].mode()[0]
train_processed2['Embarked']=train_processed1['Embarked'].fillna(mode2)
#mean=train_processed2['Age'].mean()
#train_processed2['Age']=train_processed1['Age'].fillna(mean)
train_processed2.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         0
Title            0
Groups           0
dtype: int64

In [12]:
#Convert Fare to Log Fare
train_processed2[train_processed2['Fare']!=0]['Fare']=np.log(train_processed2['Fare'])

/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
#Drop unncessary columns
train_processed2=train_processed2.set_index('PassengerId')
#Dealing with categorical data
cate_cols = train_processed2.select_dtypes(include = 'object').columns
train_processed2 = pd.get_dummies(train_processed2, columns=cate_cols)
#Drop one dummy to avoid multicollinearity
train_processed2=train_processed2.drop(columns=['Sex_female','Embarked_C','Title_ Mr'])

In [14]:
train_processed2.columns

Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Groups',
       'Sex_male', 'Embarked_Q', 'Embarked_S', 'Title_ Capt', 'Title_ Col',
       'Title_ Don', 'Title_ Dr', 'Title_ Jonkheer', 'Title_ Lady',
       'Title_ Major', 'Title_ Master', 'Title_ Miss', 'Title_ Mlle',
       'Title_ Mme', 'Title_ Mrs', 'Title_ Ms', 'Title_ Rev', 'Title_ Sir',
       'Title_ the Countess'],
      dtype='object')

In [15]:
#Predict Age for Null Value
#Building Model
train_age=train_processed2[train_processed2['Age'].notnull()]
predict=train_processed2[train_processed2['Age'].isna()]
Feature=['Pclass','SibSp', 'Parch', 'Groups','Sex_male']
X = train_age[Feature]
y = train_age['Age']
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import r2_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm
y_pred = regressor.predict(X_test)
#Predict Age for Null Age
predict.loc[:,('Age')]=regressor.predict(predict[Feature])
train_processed3=pd.concat([train_age,predict], axis=0) 
train_processed3

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Survived,Pclass,Age,SibSp,Parch,Fare,Groups,Sex_male,Embarked_Q,Embarked_S,...,Title_ Major,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,22.000000,1,0,7.2500,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,38.000000,1,0,71.2833,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,3,26.000000,0,0,7.9250,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,1,1,35.000000,1,0,53.1000,2,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,0,3,35.000000,0,0,8.0500,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,0,3,28.550285,0,0,7.2292,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
864,0,3,-6.154404,8,2,69.5500,7,0,0,1,...,0,0,1,0,0,0,0,0,0,0
869,0,3,28.550285,0,0,9.5000,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Split Age
train_processed3['AgeGroup']=np.where(train_processed3['Age'] <=15, 'Children', 
         (np.where(train_processed3['Age'] < 30, 'Adult',(np.where(train_processed3['Age'] < 50, 'Midle-Age','Elder')))))

In [17]:
#Drop one dummy to avoid multicollinearity
train_processed3 = pd.get_dummies(train_processed3, columns=['AgeGroup'])

In [18]:
train_processed4=train_processed3.drop(columns=['AgeGroup_Children'])

In [19]:
train_processed4

,Survived,Pclass,Age,SibSp,Parch,Fare,Groups,Sex_male,Embarked_Q,Embarked_S,...,Title_ Mlle,Title_ Mme,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess,AgeGroup_Adult,AgeGroup_Elder,AgeGroup_Midle-Age
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,22.000000,1,0,7.2500,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0
2,1,1,38.000000,1,0,71.2833,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1,3,26.000000,0,0,7.9250,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,1,1,35.000000,1,0,53.1000,2,0,0,1,...,0,0,1,0,0,0,0,0,0,1
5,0,3,35.000000,0,0,8.0500,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,0,3,28.550285,0,0,7.2292,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
864,0,3,-6.154404,8,2,69.5500,7,0,0,1,...,0,0,0,0,0,0,0,0,0,0
869,0,3,28.550285,0,0,9.5000,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0


**Model Training**

In [20]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [21]:
#split dataset in features and target variable
features=train_processed4.loc[:, train_processed4.columns !='Survived'].columns.tolist()
X = train_processed4[features] # Features
y = train_processed4['Survived'] # Features

In [22]:
# Split dataset into training set and test set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80% training and 20% test

In [23]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(max_depth=4)

Model 1: Decision Tree

In [24]:
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=10, random_state=None, shuffle=False) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 

KFold(n_splits=10, random_state=None, shuffle=False)


In [25]:
score=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)
    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    score.append(metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8222222222222222
Accuracy: 0.7528089887640449
Accuracy: 0.7865168539325843
Accuracy: 0.8426966292134831
Accuracy: 0.7865168539325843
Accuracy: 0.7640449438202247
Accuracy: 0.8426966292134831
Accuracy: 0.8539325842696629
Accuracy: 0.8426966292134831
Accuracy: 0.8539325842696629


In [26]:
print('Decision Tree-Average Accuracy Score:',sum(score)/len(score))

Decision Tree-Average Accuracy Score: 0.8148064918851435


Model 2: Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier()

In [28]:
score=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train random forest Classifer
    randomForest.fit(X_train,y_train)
    #Predict the response for test dataset
    y_pred = randomForest.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    score.append(metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7666666666666667
Accuracy: 0.8089887640449438
Accuracy: 0.7752808988764045
Accuracy: 0.7865168539325843
Accuracy: 0.797752808988764


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.7865168539325843
Accuracy: 0.7752808988764045
Accuracy: 0.8651685393258427
Accuracy: 0.797752808988764
Accuracy: 0.7528089887640449


In [29]:
print('Random Forest-Average Accuracy Score:',sum(score)/len(score))

Random Forest-Average Accuracy Score: 0.7912734082397004


Model 3: Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
logisticRegr = LogisticRegression()

In [31]:
score=pd.DataFrame()
kfold=[]
c=1
for train_index, test_index in kf.split(X):
    r=1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train logistic regression Classifer
    logisticRegr.fit(X_train,y_train)
    pred_proba_df = pd.DataFrame(logisticRegr.predict_proba(X_test))[1]
    threshold_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for i in threshold_list:
        y_pred = pred_proba_df.apply(lambda x: 1 if x>=i else 0)
        test_accuracy = metrics.accuracy_score(y_test,y_pred)
        score.at[r,c]=test_accuracy
        r=r+1
        y_pred = logisticRegr.predict(X_test)
        kfold.append(metrics.accuracy_score(y_test,y_pred))
    c=c+1

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [32]:
for i in range(0,9):
    print('For threshold: ',(i+1)/10)
    print('Accuracy',sum(score.iloc[i,])/10)

For threshold:  0.1
Accuracy 0.5691011235955056
For threshold:  0.2
Accuracy 0.727390761548065
For threshold:  0.3
Accuracy 0.7823595505617977
For threshold:  0.4
Accuracy 0.8159800249687889
For threshold:  0.5
Accuracy 0.8226841448189763
For threshold:  0.6
Accuracy 0.8058426966292135
For threshold:  0.7
Accuracy 0.7912484394506867
For threshold:  0.8
Accuracy 0.7507990012484395
For threshold:  0.9
Accuracy 0.6891011235955056


In [33]:
sum(kfold)/len(kfold)

0.8226841448189758

In [34]:
print('Logistic Regression-Average Accuracy Score:',sum(score)/len(score))

Logistic Regression-Average Accuracy Score: 6.111111111111111


Model 4: KNN

In [35]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
score=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train KNN Classifer
    neigh.fit(X_train,y_train)
    #Predict the response for test dataset
    y_pred = neigh.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    score.append(metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6333333333333333
Accuracy: 0.6741573033707865
Accuracy: 0.6741573033707865
Accuracy: 0.6741573033707865
Accuracy: 0.7078651685393258
Accuracy: 0.6966292134831461
Accuracy: 0.7640449438202247
Accuracy: 0.6853932584269663
Accuracy: 0.6853932584269663
Accuracy: 0.8202247191011236


In [36]:
print('KNN-Average Accuracy Score:',sum(score)/len(score))

KNN-Average Accuracy Score: 0.7015355805243446


Model 5:SVM

In [37]:
from sklearn import svm
svm = svm.SVC(decision_function_shape='ovo',gamma="auto")
score=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Train SVM Classifer
    svm.fit(X_train,y_train)
    #Predict the response for test dataset
    y_pred = svm.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    score.append(metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6444444444444445
Accuracy: 0.6966292134831461
Accuracy: 0.6966292134831461
Accuracy: 0.7303370786516854
Accuracy: 0.6853932584269663
Accuracy: 0.7528089887640449
Accuracy: 0.7640449438202247
Accuracy: 0.7752808988764045
Accuracy: 0.6966292134831461
Accuracy: 0.797752808988764


In [38]:
print('SVM-Average Accuracy Score:',sum(score)/len(score))

SVM-Average Accuracy Score: 0.7239950062421974
